In [0]:
import pandas as pd 
import matplotlib.pyplot as plt 
import io, csv, os
import numpy as np
from collections import defaultdict
import pickle
import scipy
import sklearn.metrics
from scipy.sparse.linalg import svds
import math

In [0]:
base_path = "/content/drive/My Drive/Q3 Spring 20/ML/modeling/matrix-completion/"

In [0]:
os.makedirs(base_path + '/obj/', exist_ok=True)
def save_obj(obj, name ):
    with open( base_path + '/obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(base_path + '/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [0]:
raw_df = pd.read_csv( base_path + "train.csv")
# raw_df.head(2)

In [0]:
from sklearn.utils import shuffle
raw_df = shuffle(raw_df)
raw_df = raw_df.reset_index(drop=True)

In [0]:
# split_index1 = round(0.1 * raw_df.shape[0])
split_index2 = round(0.1 * raw_df.shape[0])
validate_df = raw_df[: split_index2]

In [8]:
print(validate_df.shape) # train_df.shape

(100000, 4)


In [9]:
val_df_original = validate_df.copy()
# validate_df.head(2)

,movie-id,customer-id,rating,date
0,12470,26388,5,2005-08-16
1,9054,29008,3,2005-04-11


In [0]:
validate_df['rating'].values[:] = 0
validate_df.head(2)

In [0]:
test_df = pd.read_csv( base_path + "test.csv")
# test_df.head(2)

In [0]:
test_df['rating'].replace({"?": 0}, inplace=True)

In [0]:
combined_df = raw_df

In [14]:
# construct a dict to map users, movies with row, column ids
combined_customers = combined_df['customer-id']
combined_cid_sorted = combined_customers.sort_values()
combined_movies = combined_df['movie-id']
combined_mid_sorted = combined_movies.sort_values()

combined_cid = {}
cursor = 0
for val in combined_cid_sorted:
    if val not in combined_cid:
        combined_cid[val] = cursor
        cursor += 1

combined_mid = {}
cursor = 0
for val in combined_mid_sorted:
    if val not in combined_mid:
        combined_mid[val] = cursor
        cursor += 1

combined_movies_n = len(combined_mid)
combined_cust_n = len(combined_cid)
print(combined_movies_n, combined_cust_n)

16142 5905


In [15]:
# Make movie-by-user matrix to hold ratings
ratings = np.zeros((combined_cust_n, combined_movies_n))

for i in range(len(combined_df)):
    current = combined_df.iloc[i]
    customer_idx = combined_cid.get(current[1])
    movie_idx = combined_mid.get(current[0])
    ratings[customer_idx, movie_idx] = current[2]

ratings.shape

(5905, 16142)

In [0]:
save_obj(ratings, 'ratings_train_validate_fold')
# ratings = load_obj('ratings_train_validate_fold2')

In [0]:
means = []
f_ratings = ratings.copy()
f_ratings_t = np.transpose(f_ratings)

for i in range(f_ratings_t.shape[0]):
    record = f_ratings_t[i]
    nonzeros = record[np.nonzero(record)]
    if len(nonzeros) > 0:
        mean = np.mean(nonzeros)
    else:
        mean = 0
#     mean = mean + (np.random.rand() / 2) - 0.5
#     f_ratings_t[i] = np.where(record==0, mean, record) #type 2
    
    means.append(mean)
    record[np.nonzero(record)] = record[np.nonzero(record)] - mean # type 3
    f_ratings_t[i] = record

    # print(len(record[np.nonzero(record)]), len(record1[np.nonzero(record1)]))
f_ratings = np.transpose(f_ratings_t)

In [0]:
U1, sigma, Vt = svds(f_ratings, k = 25)

In [0]:
sigma = np.diag(sigma)

In [0]:
predicted_ratings = np.dot(np.dot(U1, sigma), Vt) + np.array(means)

In [0]:
# another fn for svd
U, s, Vh = scipy.linalg.svd(f_ratings, full_matrices=False)

In [0]:
k = 25
temp1 = U [:, :k]
temp2 = Vh [:k]
temp3 = np.diag(s[:k])

In [99]:
print(temp1.shape, temp2.shape)

(5905, 25) (25, 16142)


In [0]:
preds = np.matmul(np.matmul(temp1, temp3), temp2) # + np.array(means)

In [20]:
preds.shape

(5905, 16142)

In [0]:
val_df_original = test_df.copy()

In [0]:
mean_pred = []
gt = []
for i in range(val_df_original.shape[0]):

    temp = val_df_original.iloc[i]
    cid = combined_cid.get(temp[1])
    mid = combined_mid.get(temp[0])


    if mid is not None:
        if cid is not None:
            p = predicted_ratings[cid][mid]
        else:
            movie_r = ratings[:, mid]
            nonzeros = movie_r[np.nonzero(movie_r)]
            rating_prediction = np.mean(nonzeros)
    else:
        if cid is not None:
            movie_r = ratings[cid]
            nonzeros = movie_r[np.nonzero(movie_r)]
            rating_prediction = np.mean(nonzeros)
        else:
            rating_prediction = 3

    if temp[2] == 0:
        continue
    gt.append(temp[2])
  
    if p > 5:
        p = 5
    if p <= 0.5:
        p = 1

    mean_pred.append(round(p))

In [128]:
len(mean_pred)

250000

In [0]:
save_obj(mean_pred, 'pred_')

In [0]:
np.savetxt( base_path + 'pred_', mean_pred)

In [106]:
t_mse = ((np.array(mean_pred) - np.array(gt))**2).mean(axis=None)
t_mse

1.08234

In [89]:
c = 0
for i in range(len(mean_pred)):
    if mean_pred[i] == gt[i]:
        c += 1
c

38584

In [0]:
d = {}

for val in mean_pred:
    if val in d:
        d[val] += 1
    else:
        d[val] = 1

In [91]:
d

{1: 392, 2.0: 1709, 3.0: 35945, 4.0: 57898, 5.0: 4056}

In [0]:
gtd = {}

for val in gt:
    if val in gtd:
        gtd[val] += 1
    else:
        gtd[val] = 1

In [93]:
gtd

{1: 4637, 2: 9948, 3: 28202, 4: 33496, 5: 23717}

In [0]:
# try how it works on recommender systems packages

!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678575 sha256=22055ffacc3c33a1d0361bfb4fa0074160c261a2f8a100d81d2ac19390d60f17
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import reader

In [0]:
temp_df = raw_df[['customer-id', 'movie-id', 'rating']]
temp_df.head(2)

,customer-id,movie-id,rating
0,24294,1262,4
1,18187,1810,3


In [0]:
rdr = reader.Reader(rating_scale=(1, 5))

In [0]:
# Load the movielens-100k dataset (download it if needed).
# data = Dataset.load_builtin('ml-100k')

data = Dataset.load_from_df(temp_df, rdr)

In [0]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8997  0.8982  0.8978  0.8987  0.8985  0.8986  0.0006  
MAE (testset)     0.6994  0.6984  0.6988  0.6995  0.6988  0.6990  0.0004  
Fit time          44.10   47.32   47.31   47.38   47.23   46.67   1.29    
Test time         2.09    2.04    2.35    1.82    2.02    2.06    0.17    


{'fit_time': (44.0955810546875,
  47.32036805152893,
  47.30655908584595,
  47.384554862976074,
  47.22919726371765),
 'test_mae': array([0.69941899, 0.69843316, 0.6987851 , 0.6994929 , 0.69879845]),
 'test_rmse': array([0.89966515, 0.89817749, 0.89779095, 0.89871426, 0.89852078]),
 'test_time': (2.086087465286255,
  2.0378036499023438,
  2.3491108417510986,
  1.8214874267578125,
  2.0248587131500244)}

In [0]:
np.mean(np.array([0.89966515, 0.89817749, 0.89779095, 0.89871426, 0.89852078]))

0.8985737260000001